# Fitting of dqdv peaks
The purpose of this notebook is to evaluate and develope a robust way of fitting dqdv data. The plan is then to implement this into the cellpy.utils.ica module (as seperate classes). It would also be valuable to equip the fitting class(es) with optional ipywidgets.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import bokeh
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cellpy import cellreader
from cellpy.utils import ica
import holoviews as hv

In [ ]:
%matplotlib inline
hv.extension('bokeh')

In [ ]:
my_data = cellreader.CellpyData()
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)
my_data.load(filename)
my_data.set_mass(0.1)

In [ ]:
from lmfit.models import GaussianModel, PseudoVoigtModel, ExponentialGaussianModel, SkewedGaussianModel, LorentzianModel, SkewedVoigtModel, ConstantModel
from lmfit import CompositeModel

## Defining dqdv peak ensambles
The natural way (and the way typically seen in conferences etc) of conducting an "in-depth" ica study on a LiB cell would be to measure ica of re-buildt half-cells of both the cathode and the anode of the full cells, fit the peaks of the half cells, and then a use convolution of these fits to fit the actual full cell.


### Examples from the literature
Should include references here...

### Plan
1. Create a class (PeakEnsamble)
2. Create peak ensambles by sub-classing PeakEnsamble

## The classes

### PeakEnsamble classes

In [ ]:
class PeakEnsamble:
    """A PeakEnsamble consists of a scale and a set of peaks.
    
    The PeakEnsamble can be fitted with all the internal parameters fixed while only the scale parameter is
    varied (jitter=False), or the scale parameter is fixed while the internal parameters (individual peak heights etc)
    varied.
    
    Example:
        class SiliconPeak(PeakEnsamble):
            def __init__():
                super().__init__()
                self.name = name  # Set a prefix name
                self.prefixes = [self.name + "Scale", self.name + "01"]]
                self.peak_types = [ConstantModel, SkewedGaussianModel]
                self.a_new_variable = 12.0
                self._read_peak_definitions()
                self._init_peaks()
                
            def _read_peak_definitions(self):
                self._peak_definitions = {
                ....
                
            def _init_peaks(self):
                self._peaks = self._create_ensamble()
                self._set_hints()
                self._set_custom_hints()
                
            def _set_custom_hints(self):
                ....
        
    Attributes:
        shift (float): A common shift for all peaks in the ensamble. Should be able to fit
           this if jitter is False. TODO: chekc this up.
        name (str): Identification label that will be put in front of all peak parameter names.
        fixed (bool): 
        jitter (bool): Allow for individual fitting of the peaks in the ensamble (defaults to True).
        max_point (float): The max point of the principal peak.
        scale (float): An overall scaling parameter (fitted if jitter=False).
        sigma_p1 (float): Sigma value for the principal peak (usually the first peak). When creating the peak
            ensamble, parameters for the principal peak is set based on absolute values, while the other
            peak parameters are set based on relative values to the principal peak.
        prefixes (list): Must be set in the subclass.
        
        
    """
    def __init__(self, fixed=False, name=None, max_point=1.0, 
                 shift=0.0, sigma_p1=0.01, scale=1.0, jitter=True):
        
        self._peaks = None
        self.shift = shift
        self.name = name
        self.fixed = fixed
        self.max_point = max_point
        self.jitter = jitter
        self.scale = scale
        self.sigma_p1 = sigma_p1
        self.peak_info = dict()
        self._peak_definitions = None
        self.peak_var_names = None
        self._params = None
        
    @property
    def peaks(self):
        """lmfit.CompositeModel"""
        return self._peaks
    
    @property
    def widgets(self):
        """ipywidgets for controlling peak variables"""
        raise NotImplementedError
    
    @property
    def params(self):
        """lmfit.Parameters (OrderedDict)"""
        if self._params is None:
            self._make_params()
        return self._params
    
    def _make_params(self):
        self._params = self._peaks.make_params()
        
    def _read_peak_definitions(self):
        raise NotImplementedError("This method must be implemented when sub-classing")
        
    @property
    def peak_definitions(self):
        return self._peak_definitions
        
    def _create_ensamble(self):
        try:
            self.peak_info[self.prefixes[0]] = self.peak_types[0](prefix=self.prefixes[0])
            self.peak_info[self.prefixes[1]] = self.peak_types[1](prefix=self.prefixes[1])
        except AttributeError:
            print("you are missing peak info")
            return
        
        p = self.peak_info[self.prefixes[1]]

        for prfx, ptype in zip(self.prefixes[2:], self.peak_types[2:]):
            self.peak_info[prfx] = ptype(prefix=prfx)
            p += self.peak_info[prfx]
            
        p *= self.peak_info[self.prefixes[0]]
        return p
    
    def _set_hints(self):
        jitter = self.jitter
        if self.jitter:
            vary = True
            vary_scale = False
        else:
            vary = False
            vary_scale = True

        scale = self.scale
        
        value_dict = dict()
        peak_definitions = self.peak_definitions 
        prefix_scale = self.prefixes[0]
        prefix_peak_1 = self.prefixes[1]

        # iterate through all the peaks (not the scale) and collect variables in the value_dict
        for var_stub in peak_definitions:
            dd = peak_definitions[var_stub]
            val_1, ((frac_min, shift_min), (frac_max, shift_max)) = dd[0:2]

            v_dict = dict()
            v_dict[prefix_peak_1] = [val_1, frac_min * (val_1+shift_min), frac_max * (val_1+shift_max)]
        
            for prfx, (fact, step) in zip(self.prefixes[2:], dd[2:]):
                v_dict[prfx] = [fact * (x + step) for x in v_dict[prefix_peak_1]]

            value_dict[var_stub] = v_dict
        
        # set parameter hints based on the value_dict
        for key1 in value_dict:
            for key2 in value_dict[key1]:
                _vary = vary
                _v = value_dict[key1][key2]
                k = "".join((key2, key1))
                self._peaks.set_param_hint(k, value=_v[0], min=_v[1], max=_v[2], vary=_vary)

        # set parameter hints for scale
        self._peaks.set_param_hint("".join((prefix_scale, "c")), value=scale, min=0.1*scale, max=10*scale, vary=vary_scale)
    
    
    def _fix_full(self, prefix):
        """fixes all variables (but only for this ensamble)"""
        for k in self._params:
            if k.startswith(prefix):
                self._params[k].vary = False

class Silicon(PeakEnsamble):
    """Peak ensamble for silicon.
    
    This class is a sub-class of PeakEnsamble. Some new attributes are defined
    (in addtion to the inhereted attributes).
    
    Attributes:
        prefixes (list): A list of peak names used as the prefix when creating the peaks. The firs prefix
            should always be for the scale parameter. It is recommended not to play with this attribute.
            This attribute is required when subclassing PeakEnsamble
        peak_types (list of lmfit peak models): The length of this list must be the same as the length of the
            prefixes. It should start with a ConstantModel. This attribute is required when subclassing
            PeakEnsamble.
        crystalline (bool): Set to true if the Li3.75Si phase exists.
        
    """
    def __init__(self, scale=1.0, crystalline=False, name="Si", max_point=1000, **kwargs):
        super().__init__(sigma_p1=0.05, jitter=True, scale=scale, max_point=max_point, **kwargs)
        self.name = name
        self.prefixes = [self.name + x for x in ["Scale", "01", "02", "03"]]  # Always start with scale
        self.peak_types = [ConstantModel, SkewedGaussianModel, PseudoVoigtModel, PseudoVoigtModel]
        self.crystalline = crystalline
        self._read_peak_definitions()
        self._init_peaks()
        
    def _read_peak_definitions(self):
        self._peak_definitions = {
            "center": [
                0.25 + self.shift,          # value
                ((1.0, -0.1), (1.0, 0.1)),  # (frac-min, shift-min), (frac-max, shift-max)
                (1.0, 0.21),                # (value-frac, value-shift) between peak 1 and peak 2
                (1.0, 0.20)                 # (value-frac, value-shift) between peak 1 and peak 3
            ],
            
            "sigma": [
                self.sigma_p1,                   # value
                ((0.1, 0.0), (10.0, 0.0)),  # (frac-min, shift-min), (frac-max, shift-max)
                (1.0, 0.0), 
                (0.3, 0.0)
            ],
            
            "amplitude": [
                self.sigma_p1 * self.max_point / 0.4,   # value
                ((0.001, 0.0), (100.0, 0.0)),      # (frac-min, shift-min), (frac-max, shift-max)
                (1.0, 0.0), 
                (0.002, 0.0)
            ],
        }
        
    def _init_peaks(self):
        self._peaks = self._create_ensamble()
        self._set_hints()
        self._set_custom_hints()
        #  self._make_params()
        
    def _set_custom_hints(self):
        if not self.crystalline:
            prefix_p3 = self.prefixes[3]
            k = "".join([prefix_p3, "amplitude"])
            self._peaks.set_param_hint(k, value=0.00001, min=0.000001, vary=False)
            for n in ["center", "sigma"]:
                k = "".join([prefix_p3, n])
                self._peaks.set_param_hint(k, vary=False)  
    @property     
    def widgets(self):
        print("overrides PeakEnsamble.widgets property")
        print("because it is easier to develop this here and then copy it back to the subclass")
        
        # Need a widget set for each parameter
        # Should consist of
        #   value with max and min
        #   refine checkbox
        
        # Example
        #   Name: Si_peak01_amplitude
        #   [min] -----o--- [max] [value]
        #   Fixed: x
        
        # The ipywidgets should be coupled to the lmfit.prms somehow
        
        # Need a combined set with all the sub-widgets where the individual peak widgets are
        # greyed out if jitter is not selected
    
    
class Graphite(PeakEnsamble):
    def __init__(self, scale=1.0, name="G", **kwargs):
        super().__init__(max_point=10000.0, jitter=False, **kwargs)
        self.name = name
        self.sigma_p1 = 0.01
        self.vary = False
        self.vary_scale = True
        self.prefixes = [self.name + x for x in ["Scale", "01"]]  # Always start with scale
        self.peak_types = [ConstantModel, LorentzianModel]
        self._read_peak_definitions()
        self._init_peaks()
        
    def _read_peak_definitions(self):
        self._peak_definitions = {
            "center": [
                0.16 + self.shift,          # value
                ((1.0, -0.05), (1.0, 0.05)),  # (frac-min, shift-min), (frac-max, shift-max)
                # (1.0, 0.21),                # (value-frac, value-shift) between peak 1 and peak 2
                # (1.0, 0.20)                 # (value-frac, value-shift) between peak 1 and peak 3
            ],
            
            "sigma": [
                self.sigma_p1,                   # value
                ((0.4, 0.0), (5.0, 0.0)),  # (frac-min, shift-min), (frac-max, shift-max)
                # (1.0, 0.0), 
                # (0.3, 0.0)
            ],
            
            "amplitude": [
                self.sigma_p1 * self.max_point / 0.4,   # value
                ((0.2, 0.0), (2.0, 0.0)),      # (frac-min, shift-min), (frac-max, shift-max)
                # (1.0, 0.0), 
                # (0.002, 0.0)
            ],
        }

    def _init_peaks(self):        
        self._peaks = self._create_ensamble()
        self._set_hints()
        
   

In [ ]:
def get_widgets(parameters):
    print(parameters)
    

In [ ]:
class CompositeEnsamble(PeakEnsamble):
    def __init__(self, *ensambles, **kwargs):
        super().__init__(self, **kwargs)
        self.ensamble = ensambles
        self._peaks = None
        self._join()
        
    def _join(self):
        if len(self.ensamble) > 0:
            left = self.ensamble[0].peaks
            if len(self.ensamble) > 1:
                for ens in self.ensamble[1:]:
                    left += ens.peaks
        self._peaks = left
        
        
    def make_params(self):
        prms = self._peaks.make_params()
        print(prms)
        
    
        


In [ ]:
p = Silicon().peaks + Graphite().peaks
pars = p.make_params()

In [ ]:
p2 = CompositeEnsamble(Silicon(), Graphite())
print(p2.peaks)
print()
print(p2.params)

In [ ]:
print(p2.peaks)

In [ ]:
p.param_hints

In [ ]:
p2.peaks.param_hints

In [ ]:
pars['Si01sigma']

In [ ]:
get_widgets(pars)

In [ ]:
def fix(prefix):
    _pars = p.make_params()
    for k in _pars:
        if k.startswith(prefix):
            p[k].vary = False

In [ ]:
# initial fit
# should probably make a function out of this
cha, volt = my_data.get_ccap(12)
v, dq = ica.dqdv(volt, cha)
res = p.fit(dq, x=v, params=pars)
print("OK")

In [ ]:
# Should make a function of this

group_title = "fit"
raw = hv.Points((v, dq), label="raw", group=group_title).opts(width=1000, height=500, size=8, alpha=0.3 )
prt = {
    "ini": hv.Curve((v, res.init_fit), group=group_title).opts(alpha=0.5),
    "bes": hv.Curve((v, res.best_fit), group=group_title),
}
parts = res.eval_components()

for key in parts:
    if not key.endswith("Scale"):
        prt[key] = hv.Curve((v, parts[key]), group=group_title)

layout = raw * hv.NdOverlay(prt)
layout 

In [ ]:
print(res.fit_report())